In [ ]:
pip install ipywidgets

In [ ]:
pip install pandas

In [ ]:
# stolen from chatgpt for now, will read later and modify
# usage: answer if top frame (current frame) is different from bottom frame (last frame)
# scene change marks the first frame of a new segment
import cv2
import csv
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from pathlib import Path

# Video path and output CSV path
video_path = 'data/videos/test0_toy_lecture.mp4'
output_csv = 'annotations.csv'
sampling_frequency = 1    # second / samples per second

# Prepare CSV file
if not Path(output_csv).exists():
    with open(output_csv, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Frame', 'Label'])  # Headers: frame number and label

# Load video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS) 
frame_increment = int(fps * sampling_frequency)

# Widgets
button_change = widgets.Button(description="Scene change")
button_no_change = widgets.Button(description="No scene change")

# Initialize frame position to the last frame minus increment
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
frame = total_frames - (total_frames % fps) # initializes to final frame
last_img = cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
current_img = cap.set(cv2.CAP_PROP_POS_FRAMES, frame - frame_increment)

# Define button callback functions
def log_and_next(label):
    global frame
    # Log the label and frame number
    with open(output_csv, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([frame, label])

    # Move to next frame
    frame -= frame_increment
    display_buttons()
    display_frame()

def on_change_button_clicked(b):
    log_and_next('1')

def on_no_change_button_clicked(b):
    log_and_next('0')

# Attach event handlers
button_change.on_click(on_change_button_clicked)
button_no_change.on_click(on_no_change_button_clicked)

# Display frame
def display_frame():
    global frame, last_img, current_img

    # Check for end of video
    if frame < 0:
        print("End of Video")
        cap.release()
        return

    # create current frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame)  # Set to current frame
    ret, current_img = cap.read()
    if ret:
        # Convert to RGB and display
        current_img = cv2.cvtColor(current_img, cv2.COLOR_BGR2RGB)
        current_img = widgets.Image(
            value=cv2.imencode('.jpg', current_img)[1].tobytes(), format='jpg',
            width = '45%',
            height = '45%'
        )
    else:
        print("End of Video")

    # display
    display(current_img, last_img)
    last_img = current_img


def display_buttons():
    clear_output(wait=True)
    display(widgets.HBox([button_change, button_no_change]))

# Start the frame display
display_buttons()
display_frame()